
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## Obtain wave families
### In this case, two swell systems, one local sea

inputs required: 
  * Historical wave conditions (GOW2 hindcast)
  * Wave families sectors; the split is based on wave direction
  
in this notebook:
  * Split energy based on defined sectors
  * Remove TC associated waves to avoid double counting 


### Workflow:

<div>
<img src="resources/nb01_09.png" width="300px">
</div>


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op
from datetime import datetime

# pip
import numpy as np
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.waves import GetDistribution_gow, GetDistribution_ws

# wavespectra
from wavespectra.specdataset import SpecDataset



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')


In [3]:
# --------------------------------------
# set waves families parameters

_, TCs_r1_params = db.Load_TCs_r1_hist()   # TCs historical parameters inside big radius
_, TCs_r2_params = db.Load_TCs_r2_hist()   # TCs historical parameters inside small radius

# wave families sectors
fams_sectors = [(210, 22.5), (22.5, 135)]

# date limits for TCs removal from waves data, and TC time window (hours)
tc_rm_date1 = '1979-01-01'
tc_rm_date2 = '2019-12-31'
tc_time_window = 12



## Calculate Waves Partitions from Waves Spectra (CSIRO + wavespectra) 

In [4]:
# aux.

def fix_dir(base_dirs):
    'fix csiro direction for wavespectra (from -> to)'
    new_dirs = base_dirs + 180
    new_dirs[np.where(new_dirs>=360)] = new_dirs[np.where(new_dirs>=360)] - 360
    
    return new_dirs


In [5]:
# --------------------------------------
# load waves spectra point (CSIRO spec)

WVS_spec = db.Load_WAVES_spectra()
print(WVS_spec)
print()

# direction data fix
WVS_spec['direction'] = fix_dir(WVS_spec['direction'])

# rename variables
WVS_spec = WVS_spec.rename(
    {
        'frequency':'freq',
        'direction':'dir',
        'Efth':'efth',
    }
).set_coords({'freq','dir'})

# efth: rad to º
WVS_spec['efth'] = WVS_spec['efth'] * np.pi/180


# wavespectra parameters
wcut = 0.00000000001  # wcut = 0.3333
msw = 8
agef = 1.7

# bulk wavespectra
bulk_params = WVS_spec.spec.stats(['hs','tp','tm02','dpm','dspr'])

# partitions
ds_part = WVS_spec.spec.partition(
    WVS_spec.u10m, WVS_spec.udir, WVS_spec.depth, 
    wscut = wcut, max_swells = msw, agefac = agef,
)
WVS_parts = ds_part.spec.stats(['hs','tp','tm02','dpm','dspr']) 

# Add bulk Hs, Tp, Dir variables
WVS_parts['Hs'] = bulk_params['hs']
WVS_parts['Tp'] = bulk_params['tp']
WVS_parts['Dir'] = bulk_params['dpm']

# drop station id
WVS_parts = WVS_parts.drop('station')

# Save partitions data
db.Save_WAVES_partitions(WVS_parts)

print(WVS_parts)


<xarray.Dataset>
Dimensions:       (direction: 24, frequency: 29, string16: 16, time: 364266)
Coordinates:
    station       int32 ...
  * frequency     (frequency) float32 0.035 0.0385 ... 0.45885003 0.50473505
  * direction     (direction) float32 82.5 67.5 52.5 37.5 ... 127.5 112.5 97.5
  * string16      (string16) int32 -2147483647 -2147483647 ... -2147483647
  * time          (time) datetime64[ns] 1979-01-01 ... 2020-07-01
Data variables:
    station_name  (time) |S64 ...
    frequency1    (time, frequency) float32 ...
    frequency2    (time, frequency) float32 ...
    longitude     (time) float32 ...
    latitude      (time) float32 ...
    Efth          (time, frequency, direction) float32 ...
    depth         (time) float32 ...
    u10m          (time) float32 ...
    udir          (time) float32 ...
    curr          (time) float32 ...
    currdir       (time) float32 ...
Attributes:
    product_name:                    ww3.197901_spec.nc
    area:                           


## Calculate Historical Waves Families (CSIRO + wavespectra)

In [6]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

WVS_pts = db.Load_WAVES_partitions()       # waves partitions data (from CSIRO spectra and wavespectra toolbox)

WVS = GetDistribution_ws(WVS_pts, fams_sectors, n_partitions=8)

# Add wavespectra bulk Hs, Tp, Dir variables
WVS['Hs'] = WVS_pts['Hs']
WVS['Tp'] = WVS_pts['Tp']
WVS['Dir'] = WVS_pts['Dir']

# ensure time dimension does not repeat values
_, index = np.unique(WVS['time'], return_index=True)
WVS = WVS.isel(time=index)

print(WVS)


<xarray.Dataset>
Dimensions:      (time: 363769)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2020-07-01
Data variables:
    sea_Hs       (time) float64 nan 0.5263 0.8362 1.063 ... 0.8997 0.8783 0.8734
    sea_Tp       (time) float64 nan 2.781 3.527 4.161 ... 7.658 7.703 7.673
    sea_Dir      (time) float64 nan 68.27 70.86 73.55 ... 72.4 62.49 67.48 67.43
    swell_1_Hs   (time) float64 nan nan nan nan ... 0.295 0.2945 0.3415 0.3449
    swell_1_Tp   (time) float64 nan nan nan nan nan ... 9.092 9.107 9.537 8.92
    swell_1_Dir  (time) float64 nan nan nan nan nan ... 14.96 15.68 20.19 17.65
    swell_2_Hs   (time) float64 nan nan nan nan ... 0.03289 0.0339 0.03474
    swell_2_Tp   (time) float64 nan nan nan nan nan ... 14.57 14.54 14.52 14.51
    swell_2_Dir  (time) float64 nan nan nan nan nan ... 125.9 125.9 125.8 125.8
    Hs           (time) float64 ...
    Tp           (time) float64 ...
    Dir          (time) float64 ...



## Calculate Historical Waves Families (GOW)

In [7]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

#WVS_pts = db.Load_WAVES_partitions_GOW()  # waves partitions data (GOW)

#WVS = GetDistribution_gow(WVS_pts, fams_sectors, n_partitions=5)

# Add GOW Hs, Tp, Dir variables
#WVS['Hs'] = WVS_pts['hs']
#WVS['Tp'] = WVS_pts['tp']
#WVS['Dir'] = WVS_pts['dir']



## TCs: Waves  Selection 

In [8]:
# --------------------------------------
# Locate TCs and set category alongside WAVES data  

# remove TCs before 1979 and after 2020 (r1)
dds = TCs_r1_params.dmin_date.values[:]
ix = np.where((dds >= np.datetime64(tc_rm_date1)) & (dds <= np.datetime64(tc_rm_date2)))[0]
TCs_r1_params = TCs_r1_params.isel(storm=ix)

# select storms inside big circle 
storms_sel = TCs_r1_params.storm.values[:]

# add TCs category alongside WAVES data
WVS['TC_category'] = (('time',), np.empty(len(WVS.time))*np.nan)

for s in storms_sel:

    # waves at storm dates
    ss = TCs_r1_params.sel(storm=s)     
    wvs_s = WVS.sel(time = slice(ss.dmin_date, ss.last_date))
    
    # get hs_max date 
    t_hs_max = wvs_s.where(wvs_s.Hs ==  wvs_s.Hs.max(), drop=True).time.values[:][0]
        
    # hs_max time window 
    w1 = t_hs_max - np.timedelta64(tc_time_window,'h')
    w2 = t_hs_max + np.timedelta64(tc_time_window,'h')
    
    # set category alongside WAVES data
    ixs = np.where((WVS.time >= w1) & (WVS.time <= w2))[0]
    WVS['TC_category'][ixs] = ss.category 
    
print(WVS)

# Store historical WAVES data
db.Save_WAVES_hist(WVS)


<xarray.Dataset>
Dimensions:      (time: 363769)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2020-07-01
Data variables:
    sea_Hs       (time) float64 nan 0.5263 0.8362 1.063 ... 0.8997 0.8783 0.8734
    sea_Tp       (time) float64 nan 2.781 3.527 4.161 ... 7.658 7.703 7.673
    sea_Dir      (time) float64 nan 68.27 70.86 73.55 ... 72.4 62.49 67.48 67.43
    swell_1_Hs   (time) float64 nan nan nan nan ... 0.295 0.2945 0.3415 0.3449
    swell_1_Tp   (time) float64 nan nan nan nan nan ... 9.092 9.107 9.537 8.92
    swell_1_Dir  (time) float64 nan nan nan nan nan ... 14.96 15.68 20.19 17.65
    swell_2_Hs   (time) float64 nan nan nan nan ... 0.03289 0.0339 0.03474
    swell_2_Tp   (time) float64 nan nan nan nan nan ... 14.57 14.54 14.52 14.51
    swell_2_Dir  (time) float64 nan nan nan nan nan ... 125.9 125.9 125.8 125.8
    Hs           (time) float64 ...
    Tp           (time) float64 ...
    Dir          (time) float64 ...
    TC_category  (time) float64 nan na